In [ ]:
import glob, os
import re
import bleach
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
TARGET_CHARACTER = "ROSS"
class Line: 
    def __init__ (self, speaker, line):
        self.speaker = speaker
        self.line = line

    def __str__ (self):
        return self.speaker + ":" + self.line

In [3]:
class LinePair:
    def __init__(self, Line1, Line2):
        self.input = Line1
        self.reply = Line2

    def __str__(self):
        return "1) " + self.input.__str__() + "\n2) " + self.reply.__str__()

In [4]:
class Scene:
    def __init__ (self, conversation):
        self.charactersInScene = {}
        self.characterLines = []

        for l in conversation:
            name = l.split(":")[0]

            if name not in self.charactersInScene and name and name.upper() != "ALL" and "+" not in name:
                self.charactersInScene[name] = 0

            if name in self.charactersInScene:
                self.charactersInScene[name] += 1

        if TARGET_CHARACTER not in self.charactersInScene:
            return None

        for l in conversation:
            if ":" not in l:
                continue
            line = l.split(":")
            speaker = line[0]
            quote = re.sub("\(.*\)","",line[1])
            shorten_ellispe = re.sub("\.\.\.+","...", quote)
            if speaker in self.charactersInScene:
                self.characterLines.append(Line(speaker, shorten_ellispe))

            elif TARGET_CHARACTER in speaker or speaker.upper() == "ALL":
                self.characterLines.append(Line(TARGET_CHARACTER, shorten_ellispe))


    def __str__ (self):
        ret = self.characterLines[0].__str__()

        for l in self.characterLines[1:]:
            ret = ret + "\n" + l.__str__()

        return ret

In [13]:
def gen_line_pairs(scene):
    pairs = [] 
    if len(scene.characterLines) == 0:
        return pairs
    first = scene.characterLines[0]
    for l in scene.characterLines[1:]:
        if l.speaker == TARGET_CHARACTER and first.speaker != TARGET_CHARACTER:
            pairs.append(LinePair(first, l))
        first = l 

    return pairs

In [14]:
def shorten_pair(pair):
    input_sentences = sent_tokenize(pair.input.line)
    reply_sentences = sent_tokenize(pair.reply.line)

    new_input = ""
    num_input = 0
    for s in reversed(input_sentences):
        s_tokens = word_tokenize(s)
        if num_input + len(s_tokens) <= 30:
            new_input = s + " " + new_input
            num_input += len(s_tokens)

        else:
            break

    new_input = new_input.strip()

    new_reply = ""
    num_reply = 0
    for s in reply_sentences:
        s_tokens = word_tokenize(s)
        if num_reply + len(s_tokens) <= 30:
            new_reply = new_reply + " " + s
            num_reply += len(s_tokens)
        else:
            break

    new_reply = new_reply.strip()
    
    pair.input.line = new_input
    pair.reply.line = new_reply

In [22]:
def process_file(file_name):
    f = open(file_name, 'r', encoding = "ISO-8859-1")
    f_contents = f.read()
    f.close()
    scenes = re.compile("\[.*\]").split(f_contents)

    all_pairs = []
    for scene in scenes:
        lines_empty = scene.split("\n")
        
        sc = Scene(lines_empty)
        line_pairs = gen_line_pairs(sc)

        for p in line_pairs:
            shorten_pair(p)
            all_pairs.append(p)
            print("hi")

    for p in all_pairs:
        print(p)
        
process_file("scripts/0417.txt")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [15]:
#for s in os.listdir("scripts/"):    